In [1]:
# %% [markdown]
# ## 🚀 SFO 프로젝트- 모델 학습 및 저장
#
# - **전처리 완료된 CSV 파일 로드**
# - Train/Validation 데이터 분할 (시간 기반)
# - **LightGBM 베이스라인 모델 학습**
# - **검증 데이터에 대한 예측 및 MAE, R2 출력**
# - **학습된 모델 파일로 저장**
#
# **[시작 전 준비사항]**
# 1. `00_baseline_processed` 스크립트를 먼저 실행하여 `baseline_processed.csv` 파일이 생성되어 있어야 합니다.
# 2. 필요한 라이브러리 설치: `pip install pandas numpy lightgbm scikit-learn joblib`
#

import pandas as pd
import numpy as np
import os
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, r2_score
import joblib # 모델 저장을 위한 라이브러리

# --- 0. 설정 및 파일 경로 정의 ---
input_processed_csv_file = '../data/baseline_processed.csv' 
model_save_path = '../data/baseline_lightgbm_model.pkl' # 학습된 모델 저장 경로

target_column = 'T일 예정 수주량'

print("--- [SFO 프로젝트 [2/2] - 모델 학습 및 저장 시작] ---")


# --- 1단계: 전처리된 데이터 로드 ---
print(f"\n--- [1단계: 전처리된 데이터 '{input_processed_csv_file}' 로드 시작] ---")
df_manual_features_encoded = None
try:
    if not os.path.exists(input_processed_csv_file):
        raise FileNotFoundError(f"🚨 오류: 입력 파일 '{input_processed_csv_file}'을(를) 찾을 수 없습니다. \n   [파일 1: 01_preprocess_and_save.py] 스크립트를 먼저 실행했는지 확인하세요.")
    
    df_manual_features_encoded = pd.read_csv(input_processed_csv_file)
    
    # !! 매우 중요 !!
    # CSV로 저장하면서 문자열이 된 'Date' 컬럼을 다시 datetime 객체로 변환합니다.
    # 시간 기반 데이터 분할(splitting)을 위해 필수입니다.
    df_manual_features_encoded['Date'] = pd.to_datetime(df_manual_features_encoded['Date'])
    
    print(f"✅ 전처리된 데이터 로드 완료 (shape: {df_manual_features_encoded.shape})")
    print(f"   'Date' 컬럼 타입: {df_manual_features_encoded['Date'].dtype}")

except FileNotFoundError as e:
    print(f"🚨 {e}")
except Exception as e:
    print(f"🚨 1단계 데이터 로드 중 오류: {e}")
    df_manual_features_encoded = None


--- [SFO 프로젝트 [2/2] - 모델 학습 및 저장 시작] ---

--- [1단계: 전처리된 데이터 '../data/baseline_processed.csv' 로드 시작] ---
✅ 전처리된 데이터 로드 완료 (shape: (10624, 19))
   'Date' 컬럼 타입: datetime64[ns]


In [2]:
# --- 2단계: LightGBM 최종 모델 학습 및 예측 ---
final_model = None

if df_manual_features_encoded is not None and not df_manual_features_encoded.empty:
    print("\n--- [2단계: LightGBM 최종 모델 학습 및 예측 시작] ---")
    try:
        # 2.1. 데이터 분할 (Train/Validation - 시간 기반)
        # 'Date' 컬럼이 datetime 타입이어야 sort_values가 올바르게 작동합니다.
        df_manual_features_encoded = df_manual_features_encoded.sort_values(by='Date')
        
        train_ratio = 0.8
        split_idx = int(len(df_manual_features_encoded) * train_ratio)

        train_df = df_manual_features_encoded.iloc[:split_idx]
        val_df = df_manual_features_encoded.iloc[split_idx:]

        # ▼▼▼▼▼ [수정된 부분] ▼▼▼▼▼
        # 'object' 타입의 원본 Product_Number 컬럼을 피처에서 제외시킵니다.
        features_to_exclude_from_X = ['Date', target_column, 'Product_Number']
        # ▲▲▲▲▲ [수정된 부분] ▲▲▲▲▲

        X_train = train_df.drop(columns=features_to_exclude_from_X)
        y_train = train_df[target_column]
        X_val = val_df.drop(columns=features_to_exclude_from_X)
        y_val = val_df[target_column]
        
        # (참고) 만약 'DayOfWeek' 원본 컬럼도 남아있다면 위 리스트에 'DayOfWeek'도 추가해야 할 수 있습니다.
        # features_to_exclude_from_X = ['Date', target_column, 'Product_Number', 'DayOfWeek']

        print(f"2.1. 데이터 분할 완료: Train_X: {X_train.shape}, Train_y: {y_train.shape}, Val_X: {X_val.shape}, Val_y: {y_val.shape}")
        print(f"   - 최종 모델 학습에 사용될 피처 개수: {X_train.shape[1]}개")


        # 2.2. LightGBM 최종 모델 학습 (Optuna 최적 파라미터 적용)
        print("\n2.2. LightGBM 최종 모델 학습 시작 (Optuna 최적 파라미터)...")
        # --- 여기에 Optuna로 찾은 최종 최적 파라미터를 입력하세요! ---
        # 예시 파라미터입니다. 발표의 MAE 11.92, R2 0.957을 달성한 파라미터로 교체해야 합니다.
        lgbm_final_params = {
            'objective': 'regression_l1', # MAE를 직접 최적화
            'metric': 'mae',
            'n_estimators': 2000,         # 충분히 큰 값 설정 후 early stopping
            'learning_rate': 0.01,
            'num_leaves':32,
            'max_depth': 7,
            'min_child_samples': 15,
            'subsample': 0.7,
            'colsample_bytree': 0.7,
            'random_state': 42,
            'n_jobs': -1,
            'verbose': -1,                # 학습 과정 메시지 출력 억제
            'boosting_type': 'gbdt',
        }

        final_model = lgb.LGBMRegressor(**lgbm_final_params)
        final_model.fit(X_train, y_train,
                        eval_set=[(X_val, y_val)],
                        eval_metric='mae',
                        callbacks=[lgb.early_stopping(100, verbose=False)]) # 100회 동안 개선 없으면 조기 종료

        print("\n✅ LightGBM 최종 모델 학습 완료!")

        # 2.3. 검증 데이터에 대한 예측 및 성능 평가
        val_preds = final_model.predict(X_val)
        final_mae = mean_absolute_error(y_val, val_preds)
        final_r2 = r2_score(y_val, val_preds)
        print(f"   - 최종 모델 검증 MAE: {final_mae:.2f}")
        print(f"   - 최종 모델 검증 R2: {final_r2:.3f}")

        # 2.4. 학습된 모델 저장
        try:
            joblib.dump(final_model, model_save_path)
            print(f"✅ 학습된 최종 모델이 '{model_save_path}'에 저장되었습니다.")
        except Exception as e:
            print(f"🚨 모델 저장 중 오류 발생: {e}")

        # 2.5. (선택 사항) 저장된 모델 불러오기 예시
        # loaded_model = joblib.load(model_save_path)
        # print(f"✅ 모델 '{model_save_path}' 불러오기 완료.")
        # loaded_preds = loaded_model.predict(X_val)
        # print(f"   - 불러온 모델로 예측한 MAE: {mean_absolute_error(y_val, loaded_preds):.2f}")


    except Exception as e:
        print(f"🚨 2단계 LightGBM 모델 학습 및 예측 중 오류 발생: {e}")
else:
    print("🚨 1단계 데이터 로드 실패로 2단계 모델 학습 및 예측을 시작할 수 없습니다.")

print("\n--- [SFO 프로젝트 [2/2] - 모델 학습 및 저장 종료] ---")


--- [2단계: LightGBM 최종 모델 학습 및 예측 시작] ---
2.1. 데이터 분할 완료: Train_X: (8499, 16), Train_y: (8499,), Val_X: (2125, 16), Val_y: (2125,)
   - 최종 모델 학습에 사용될 피처 개수: 16개

2.2. LightGBM 최종 모델 학습 시작 (Optuna 최적 파라미터)...

✅ LightGBM 최종 모델 학습 완료!
   - 최종 모델 검증 MAE: 16.75
   - 최종 모델 검증 R2: 0.928
✅ 학습된 최종 모델이 '../data/baseline_lightgbm_model.pkl'에 저장되었습니다.

--- [SFO 프로젝트 [2/2] - 모델 학습 및 저장 종료] ---
